In [55]:
import sys
import os
import json
from pprint import pprint

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(project_root)

import simulate_routes as sr
import models.vehicle_models.energy_consumption as ec

In [56]:
with open('../models/vehicle_models/vehicle_data.json', 'r') as f:
    vehicle_data = json.load(f)

with open('../models/vehicle_models/static_data.json', 'r') as f:
    static_data = json.load(f)

with open('../models/vehicle_models/battery_data.json', 'r') as f:
    battery_data = json.load(f)

In [57]:
route_dict = {'nodes': [6058290567, 1692444945],
             'osmid': '36996649',
             'section1': {'acceleration': 0,
                          'avg_incline_angle': 3.16,
                          'climb': 0.6933822631835938,
                          'coords': [-2.6047615,
                                     51.4586801,
                                     -2.6046479,
                                     51.4586822],
                          'distance': 12.563668839179291,
                          'points': ['point1', 'point2'],
                          'velocity': 5.6},
             'section2': {'acceleration': 0,
                          'avg_incline_angle': False,
                          'climb': False,
                          'coords': [-2.6046479,
                                     51.4586822,
                                     -2.6044894999999997,
                                     51.4586847],
                          'distance': 17.51753471986847,
                          'points': ['point2', 'point3'],
                          'velocity': 5.6},
             'section2_1': {'acceleration': 0,
                            'avg_incline_angle': False,
                            'climb': False,
                            'coords': [-2.6044894999999997,
                                       51.4586847,
                                       -2.6043311,
                                       51.4586872],
                            'distance': 17.51753471986847,
                            'points': ['point2', 'point3'],
                            'velocity': 5.6},
             'section3': {'acceleration': 0,
                          'avg_incline_angle': 1.22,
                          'climb': 0.1400909423828125,
                          'coords': [-2.6043311,
                                     51.4586872,
                                     -2.6042717,
                                     51.4586881],
                          'distance': 6.569010640492013,
                          'points': ['point3', 'point4'],
                          'velocity': 5.6},
             'section4': {'acceleration': 0,
                          'avg_incline_angle': False,
                          'climb': False,
                          'coords': [-2.6042717,
                                     51.4586881,
                                     -2.6041266,
                                     51.45868085],
                          'distance': 16.064903837773407,
                          'points': ['point4', 'point5'],
                          'velocity': 5.6},
             'section4_1': {'acceleration': 0,
                            'avg_incline_angle': False,
                            'climb': False,
                            'coords': [-2.6041266,
                                       51.45868085,
                                       -2.6039815,
                                       51.4586736],
                            'distance': 16.064903837773407,
                            'points': ['point4', 'point5'],
                            'velocity': 5.6},
             'section5': {'acceleration': 0,
                          'avg_incline_angle': 1.65,
                          'climb': 0.5071640014648438,
                          'coords': [-2.6039815,
                                     51.4586736,
                                     -2.6038228,
                                     51.4586807],
                          'distance': 17.566254536710172,
                          'points': ['point5', 'point6'],
                          'velocity': 5.6},
             'smooth': True}

In [58]:
consumptions = []
distances = []
climbs = []
    
# Target velocity (constant across the route)
target_velocity = vehicle_data["max_speed"]

    # Starting velocity for each path
current_velocity = 0  # Always start a new path from zero


# Sort sections if needed (assuming they're already in order)
# Process each section in the path
for i, section_data in route_dict.items():
    if "section" in i: 
        # Determine if this is a start from zero
        if i == 0 or not route_dict["smooth"]:
            start_from_zero = True
        else:
            start_from_zero = False
        print(start_from_zero)
        
        # Reset velocity if starting from zero
        if start_from_zero:
            current_velocity = 0
        
        print(current_velocity)        # Process section with acceleration model
        segment_result = ec.calculate_segment_energy_with_acceleration(
            current_velocity, target_velocity, section_data,
            vehicle_data, static_data, vehicle_data["max_motor_power"], vehicle_data["motor_eff"]
        )
        # Update current velocity for next section
        current_velocity = segment_result["final_velocity"]
        
        # Extract results
        if "total_energy" in segment_result:
            energy = segment_result["total_energy"]
        else:
            energy = segment_result["energy_consumption"]
        
        # Append to results
        distances.append(section_data['distance'])
        consumptions.append(energy)
        climbs.append(section_data['climb'])

        pprint(segment_result)
        # Optional: Store detailed results in the original data structure
        # route_dict[path][section]["energy_details"] = segment_result
total_distance = sum(distances)
total_consumption = sum(consumptions)
total_climb = sum(climbs)

False
0
{'acceleration': 1.2480430836494636,
 'energy_consumption': np.float64(2.222605370084617),
 'final_velocity': np.float64(5.6),
 'initial_velocity': 0,
 'reached_target': np.True_,
 'time': np.float64(4.487024585421175)}
False
5.6
{'acceleration': 0,
 'energy_consumption': np.float64(0.4566746608648057),
 'final_velocity': np.float64(5.6),
 'initial_velocity': np.float64(5.6),
 'reached_target': False,
 'time': np.float64(3.1281311999765125)}
False
5.6
{'acceleration': 0,
 'energy_consumption': np.float64(0.4566746608648057),
 'final_velocity': np.float64(5.6),
 'initial_velocity': np.float64(5.6),
 'reached_target': False,
 'time': np.float64(3.1281311999765125)}
False
5.6
{'acceleration': 0,
 'energy_consumption': np.float64(0.2982833632608333),
 'final_velocity': np.float64(5.6),
 'initial_velocity': np.float64(5.6),
 'reached_target': False,
 'time': np.float64(1.1730376143735737)}
False
5.6
{'acceleration': 0,
 'energy_consumption': np.float64(0.41880519315425524),
 'final_